# Steady-State Thermal Model of META

Here's the setup:

![](images/meta-steady-state-analysis-config.png)

There's two nodes, the body (i.e. primary chassis) and the solar panels. For each, the equations boil down to:

$$
Q_\text{net} = 0 = Q_\text{sun} + Q_\text{ground} + Q_\text{sky} + Q_\text{conduction} + Q_\text{gen}
$$

where

$$
\begin{align*}
Q_\text{sun} &= \alpha A G_\text{SC} \\
Q_{\text{rad},i} &= F_i A_i \epsilon_i \sigma (T_i^4 - T^4)
\end{align*}
$$

where $G_\text{SC}$ is the incidence solar flux at Mercury (assumed to be worst case at perihelion), and $F_i$ is the view-factor from the source of $T_i$. The above expression for $Q_i$ is useful for radiative heat transfer to/from $Q_\text{ground}$ and $Q_\text{sky}$. Note that if there are multiple faces exposed to a temperature (as is in our case), we need to sum over all the various face areas, view factors, and materials for the total. $Q_\text{gen}$ is the internally generated power, perhaps from active components. Finally, $Q_\text{conduction}$ is the conductive heat transfer between the body and solar panels, via a thermal interface between the two. Conduction follows the following law:

$$
Q_{1\rightarrow2} = \frac{k A}{L} \left( T_1 - T_2 \right)
$$

In [1]:
from scipy.optimize import root


def DEGC_TO_K(degC: float) -> float:
    return degC + 273.15


def K_TO_DEGC(K: float) -> float:
    return K - 273.15


SOLAR_FLUX = 14462      # [W/m^2], Maximum at perihelion
STEF_BOLTZ = 5.6703e-8  # [W/(m^2 K^4)]

# View Factors
F_GROUND_TO_BOTTOM = 1
F_GROUND_TO_VERT = 0.5
F_SKY_TO_TOP = 1
F_SKY_TO_VERT = 0.5

In [2]:
# Temperatures [K]
T_GROUND = DEGC_TO_K(50)
T_SKY = 3

# Material Properties
# fmt: off
# Body
ABSORPTIVITY_BODY_FRONT     = 0.15      # Conservative value for white paint on chassis front
EMISSIVITY_BODY_FRONT       = 0.88      # Estimate for chassis front
EMISSIVITY_BODY_BOTTOM      = 0.8       # Radiator on bottom
EMISSIVITY_BODY_TOP         = 0.8       # Radiator on top
EMISSIVITY_BODY_BACKSIDE    = 0.8       # Radiator on sides and back
# Panels
ABSORPTIVITY_PANEL_FRONT    = 0.75      # IME - CONDUCTIVE COATING
EMISSIVITY_PANEL_FRONT      = 0.79      # IME - CONDUCTIVE COATING
EMISSIVITY_PANEL_BACK       = 0.985     # Radiator on back of solar panels
EFFICIENCY_PANEL            = 0.20      # Estimate of how much absorbed energy is used for power
# fmt: on

# Body Dimensions [m]
X_DIM = 0.3
Y_DIM = 0.2
Z_DIM = 0.2

# Body Areas [m^2]
A_BODY_FRONT = Y_DIM * Z_DIM
A_BODY_BOTTOM = X_DIM * Y_DIM
A_BODY_TOP = A_BODY_BOTTOM
A_BODY_BACKSIDE = 2 * (X_DIM * Z_DIM) + (Y_DIM * Z_DIM)

# Panel Area [m^2]
A_PANEL = 2 * (0.2 * 0.3)

# Thermal Conduction Interface
K_INTERFACE = 0  # [W/(m K)]
A_INTERFACE = 2 * Z_DIM * 0.005  # [m^2]
L_INTERFACE = 0.01  # [m]
KAPPA_INTERFACE = K_INTERFACE * A_INTERFACE / L_INTERFACE  # [W/K]

def Q_net(T_body: float, T_panel: float) -> "tuple[float, float]":
    """Compute net heat transfer out of chassis and solar panels.

    Inputs:
        T_body: Temperature of body in [degC]
        T_panel: Temperature of solar panels in [degC]

    Returns:
        Two net heat transfers (positive is leaving) in [W], for body and panels
    """

    T_body = DEGC_TO_K(T_body)
    T_panel = DEGC_TO_K(T_panel)

    def Q_rad_body() -> float:
        """Calculate net radiation leaving body / chassis"""
        # fmt: off
        C_GROUND = STEF_BOLTZ * (
            A_BODY_BOTTOM   * F_GROUND_TO_BOTTOM    * EMISSIVITY_BODY_BOTTOM    +
            A_BODY_BACKSIDE * F_GROUND_TO_VERT      * EMISSIVITY_BODY_BACKSIDE  +
            A_BODY_FRONT    * F_GROUND_TO_VERT      * EMISSIVITY_BODY_FRONT
        )
        C_SKY = STEF_BOLTZ * (
            A_BODY_TOP      * F_SKY_TO_TOP  * EMISSIVITY_BODY_TOP       +
            A_BODY_BACKSIDE * F_SKY_TO_VERT * EMISSIVITY_BODY_BACKSIDE  +
            A_BODY_FRONT    * F_SKY_TO_VERT * EMISSIVITY_BODY_FRONT
        )
        # fmt: on
        Q_sun = ABSORPTIVITY_BODY_FRONT * A_BODY_FRONT * SOLAR_FLUX
        Q_ground = C_GROUND * (T_GROUND**4 - T_body**4)
        Q_sky = C_SKY * (T_SKY**4 - T_body**4)
        return Q_sun + Q_ground + Q_sky

    def Q_rad_panel() -> float:
        """Calculate net radiation leaving solar panel assembly"""
        # fmt: off
        C_GROUND = STEF_BOLTZ * A_PANEL * F_GROUND_TO_VERT * (
            EMISSIVITY_PANEL_FRONT + EMISSIVITY_PANEL_BACK
        )
        C_SKY = STEF_BOLTZ * A_PANEL * F_SKY_TO_VERT * (
            EMISSIVITY_PANEL_FRONT + EMISSIVITY_PANEL_BACK
        )
        # fmt: on
        Q_sun = ABSORPTIVITY_PANEL_FRONT * (1 - EFFICIENCY_PANEL) * A_PANEL * SOLAR_FLUX
        Q_ground = C_GROUND * (T_GROUND**4 - T_panel**4)
        Q_sky = C_SKY * (T_SKY**4 - T_panel**4)
        return Q_sun + Q_ground + Q_sky

    # Compute radiative heat transfer [W]
    Q_body_net = Q_rad_body()
    Q_panel_net = Q_rad_panel()

    # Compute conduction between body and panel
    Q_panel_to_body = KAPPA_INTERFACE * (T_panel - T_body)
    Q_body_net += Q_panel_to_body
    Q_panel_net -= Q_panel_to_body

    return Q_body_net, Q_panel_net

In [3]:
sol = root(lambda x: Q_net(*x), x0=(50, 50))
assert sol.success
T_body, T_panel = sol.x

print(f"T_body  = {T_body:>7.2f} degC")
print(f"T_panel = {T_panel:>7.2f} degC")

T_body  =   53.29 degC
T_panel =  277.09 degC
